In [3]:
import logging
import os
import pymongo
import cobra
import cobrakbase
import time
import copy
import json
import pandas as pd
logger = logging.getLogger(__name__)

cobrakbase 0.2.7


In [2]:
%run ../../../annotation-server/annotation_api_neo4j.py
host, port, user, pwd = ("0.0.0.0", 7687, "neo4j", "123585")
def init_annotation_api(host, port, user, pwd):
    annotation_api = AnnotationApiNeo4j(user=user, pwd=pwd, port=port, host=host)
    annotation_api.neo4j_graph = Graph("http://neo4j:" + pwd + "@" + host + ":7474")
    annotation_api.matcher = NodeMatcher(annotation_api.neo4j_graph)
    annotation_api.r_matcher = RelationshipMatcher(annotation_api.neo4j_graph)
    annotation_api.init_constraints()
    return annotation_api

annotation_api = init_annotation_api(host, port, user, pwd)

cobrakbase 0.2.5


In [14]:
token = "DGXHUU34PUCT2V5JGUYA23P6TMQJM3FK"
kbase = cobrakbase.KBaseAPI(token)

In [5]:
genome_ids = annotation_api.get_genome_set('ModelSEED2')

In [51]:
for genome_id in genome_ids:
    genome_node = annotation_api.get_genome(genome_id)
    taxon_ref = genome_node['taxon_ref']
    ref = kbase.get_object_info_from_ref(taxon_ref)
    taxon = kbase.get_object(ref.id, ref.workspace_id)
    annotation_api.update_node('RefSeqGenome', genome_id, {'taxonomy_id' : str(taxon['taxonomy_id'])})

In [9]:
kegg_genomes = None
with open('/Users/fliu/workspace/jupyter/python3/data/kegg_genomes.json', 'r') as fh:
    kegg_genomes = json.loads(fh.read())

In [20]:
tax_to_kegg_genome = {}
for genome_id in kegg_genomes:
    gn = kegg_genomes[genome_id]
    t = gn['taxonomy'].split()[0]
    if t.startswith('TAX:'):
        tax_to_kegg_genome[gn['taxonomy'].split()[0][4:]] = genome_id
    else:
        print('!')

In [19]:
gcf_to_kegg = {}
kegg_to_gcf = {}
for genome_id in genome_ids:
    genome_node = annotation_api.get_genome(genome_id)
    if genome_node['taxonomy_id'] in tax_to_kegg_genome:
        kegg_gn_id = tax_to_kegg_genome[genome_node['taxonomy_id']]
        gcf_to_kegg[genome_id] = kegg_gn_id
        kegg_to_gcf[kegg_gn_id] = genome_id

NameError: name 'genome_ids' is not defined

In [108]:
genome_node = annotation_api.get_genome(genome_id)
genome_node

<Node id=507764 labels={'RefSeqGenome'} properties={'genetic_code': 11, 'notes': '', 'release': '1', 'created_at': 1570317559501, 'scientific_name': '[Clostridium] clariflavum DSM 19732', 'taxonomy': 'cellular organisms; Bacteria; Terrabacteria group; Firmicutes; Clostridia; Clostridiales; Ruminococcaceae; Ruminiclostridium; [Clostridium] clariflavum', 'source': 'refseq', 'type': 'Representative', 'external_source_origination_date': '18-AUG-2015', 'dna_size': 4897678, 'reference_annotation': 0, 'taxonomy_id': '720554', 'original_source_file_name': 'GCF_000237085.1_ASM23708v1_genomic.gbff', 'updated_at': 1570317559501, 'num_contigs': 1, 'assembly_ref': '19217/203793/1', 'domain': 'Bacteria', 'taxon_ref': '12570/600800/1', 'id': 'GCF_000237085.1.RAST', 'source_id': 'NC_016627', 'gc_content': 0.357, 'genbank_handle_ref': 'KBH_1247671', 'key': 'GCF_000237085.1', 'md5': '836005f5afe9a7dc3b2397722e999ecc'}>

In [109]:
assembly_ref = genome_node['assembly_ref']
ref = kbase.get_object_info_from_ref(assembly_ref)
assembly = kbase.get_object(ref.id, ref.workspace_id)

In [4]:
refseq_assembly = None
with open('/Users/fliu/workspace/jupyter/python3/data/refseq_assembly.json', 'r') as fh:
    refseq_assembly = json.loads(fh.read())

In [7]:
for k in refseq_assembly:
    a = refseq_assembly[k]

In [29]:
count = {}
assembly_ids = set()
for gn_id in kegg_genomes:
    gn = kegg_genomes[gn_id]    
    lineage = gn['lineage'].split(';')[0]
    if lineage == 'Bacteria' or lineage == 'Archaea':
        assembly_id = get_assembly_id(gn)
        assembly_ids.add(assembly_id)
    if lineage not in count:
        count[lineage] = 0
    count[lineage] += 1
count

{'Bacteria': 5772, 'Archaea': 325, 'Eukaryota': 541, 'Viruses': 341}

In [27]:
print(get_assembly_id(gn))
gn

GCA_000751035.1


{'id': None,
 'entry': 'T03289',
 'name': 'sepp, 1449752',
 'source': None,
 'version': None,
 'description': '',
 'names': None,
 'definition': 'Staphylococcus epidermidis PM221',
 'properties': {},
 'dataSource': 'GenBank (Assembly:GCA_000751035.1)\n            BioProject:258266',
 'annotation': 'yes',
 'lineage': 'Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcaceae; Staphylococcus',
 'taxonomy': 'TAX:1449752\n  LINEAGE   Bacteria; Firmicutes; Bacilli; Bacillales; Staphylococcaceae; Staphylococcus'}

In [25]:
def get_assembly_id(gn):
    assembly_id = gn['dataSource'].split()[1].split(':')[1]
    if assembly_id.endswith(')'):
        assembly_id = assembly_id[:len(assembly_id) - 1]
    return assembly_id

In [26]:
for i in refseq_assembly:
    if i.startswith('GCA'):
        genbank_assembly = refseq_assembly[i]
        break

In [145]:
genbank_assembly['submitterorganization']
genbank_assembly['synonym']['genbank']
genbank_assembly['synonym']['refseq']
genbank_assembly['synonym']['similarity']

{'genbank': 'GCA_000177535.2',
 'refseq': 'GCF_000177535.2',
 'similarity': 'identical'}

In [18]:
for gcf_id in gcf_to_kegg:
    genome_id = gcf_to_kegg[gcf_id]
    try:
        genome = kegg_genomes[genome_id]
        genome_token = genome['name'].split(',')[0]
        assembly_id = genome['dataSource'].split()[1].split(':')[1]
        if assembly_id.endswith(')'):

            assembly_id = assembly_id[:len(assembly_id) - 1]
        if assembly_id.startswith('GCA'):
            print(assembly_id)
            pass
    except Exception:
        print(genome_id)
        break

NameError: name 'gcf_to_kegg' is not defined

'GCA_009734465.1)'

In [149]:
def report_genomes(gcf_to_kegg):
    data = {
        'gcf' : [],
        'kegg' : [],
        'gcf_name' : [],
        'kegg_name' : [],
        'gcf_l' : [],
        'kegg_l' : [],
        'kegg_data_source' : [],
        'submitterorganization' : [],
        'genbank' : [],
        'refseq' : [],
        'genbank/refseq' : [],
    }
    for gcf_id in gcf_to_kegg:
        kegg_gn_id = gcf_to_kegg[gcf_id]
        data['gcf'].append(gcf_id)
        data['kegg'].append(kegg_gn_id)
        
        genome_node = annotation_api.get_genome(gcf_id)
        kegg_genome = kegg_genomes[kegg_gn_id]
        kegg_data_source = kegg_genomes[kegg_gn_id]['dataSource'].split()[1]
        kegg_lineage = kegg_genome['taxonomy'].split('\n')[1][12:]
        data['gcf_name'].append(genome_node['scientific_name'])
        data['kegg_name'].append(kegg_genome['definition'])
        data['gcf_l'].append(genome_node['taxonomy'])
        data['kegg_l'].append(kegg_lineage)
        data['kegg_data_source'].append(kegg_data_source)
        
        submitterorganization = None
        genbank = None
        refseq = None
        genbank_refseq = None
        
        assembly_id = kegg_data_source.split(':')[1]
        if assembly_id.endswith(')'):
            assembly_id = assembly_id[:len(assembly_id) - 1]
        if assembly_id in refseq_assembly:
            assembly = refseq_assembly[assembly_id]
            if assembly is not None:
                submitterorganization = assembly['submitterorganization']
                genbank = assembly['synonym']['genbank']
                refseq = assembly['synonym']['refseq']
                genbank_refseq = assembly['synonym']['similarity']
            else:
                print(assembly_id)
        data['submitterorganization'].append(submitterorganization)
        data['genbank'].append(genbank)
        data['refseq'].append(refseq)
        data['genbank/refseq'].append(genbank_refseq)
    df = pd.DataFrame(data)
    df = df.set_index('gcf')
    return df
df = report_genomes(gcf_to_kegg)


GCA_000146065.1


In [151]:
df.to_csv('/Users/fliu/workspace/jupyter/python3/data/report_annotation_genomes_with_kegg.tsv', sep='\t')

In [ ]:
def get_gene_function(gene_id, genome_id, source = 'BETA_RAST'):
    node_gene = annotation_api.get_node('KBaseGene', gene_id + '@' + genome_id)
    if node_gene == None:
        logger.warning('gene not found %s %s', gene_id, genome_id)
        return None
    node_gene = annotation_api.matcher.get(node_gene.id)
    gfunctions = set()
    for rel in annotation_api.neo4j_graph.match((node_gene, ), r_type="has_annotation", ):
        if source in rel['function_source']:
            function = Neo4jAnnotationFunction(rel.end_node)
            gfunctions.add(function.value)
    return gfunctions

In [38]:
#annotation_api.matcher.match
n = annotation_api.matcher.get(genome_node.id)
for r in annotation_api.neo4j_graph.match((n, None), r_type=None,limit=None):
    if 'has_gene' not in r.types():
        print(r)

True